# Introduction

# Imortant Libraries

In [7]:
! pip install contractions
! pip install emoji
! pip install arabic_reshaper python-bidi nltk snowballstemmer

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import plotly.express as px

In [9]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from sklearn.base import BaseEstimator, TransformerMixin
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji
import arabic_reshaper
from bidi.algorithm import get_display
from snowballstemmer import stemmer
from imblearn.over_sampling import SMOTE

In [10]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Dataset

In [2]:
# read data
dialects_data = pd.read_csv('/kaggle/input/dialects-db/dialects_data.csv', engine='python')

In [3]:
# show the first five rows of the dataframe
dialects_data.head()

,id,text,dialect
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY


# EDA and Preprocessing


In [14]:
dialects_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147734 entries, 0 to 147733
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       147734 non-null  object
 1   text     147732 non-null  object
 2   dialect  147718 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


## check NaNs

In [15]:
dialects_data.isna().sum()

id          0
text        2
dialect    16
dtype: int64

**we will delete rows with missing values**

In [16]:
# drop rows with missing values
dialects_data = dialects_data.dropna()

In [17]:
#check for missing values ag
dialects_data.isna().sum()

id         0
text       0
dialect    0
dtype: int64

## check duplicates
we need drop id column before check duplicates

In [18]:
# Drop the 'id' column
dialects_data = dialects_data.drop(columns=['id'])

In [19]:
dialects_data.head()

,text,dialect
0,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY


In [20]:
# Check for duplicates
duplicates = dialects_data.duplicated()

# Print the number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 0


## check dataset balancing

In [21]:
# Plot the distribution of 'dialect' column with colorful bars
fig = px.histogram(dialects_data, x='dialect', color='dialect',
                   title='Distribution of Dialects',
                   labels={'dialect': 'Dialect', 'count': 'Frequency'},
                   color_discrete_sequence=px.colors.qualitative.Set1)
fig.update_xaxes(categoryorder='total descending')
fig.update_layout(xaxis_title="Dialect", yaxis_title="Frequency")
fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


## **Ooh there is big imbalance here**
We Can deal with that in difference ways.

Here are a few strategies to handle imbalanced datasets:

*    **Resampling Techniques:**
        Over-sampling: Increase the number of minority class samples by randomly duplicating them or generating synthetic samples (e.g., using SMOTE).
        Under-sampling: Decrease the number of majority class samples by randomly removing them.
        Combined over- and under-sampling: A combination of over-sampling the minority class and under-sampling the majority class.

*    **Algorithmic Techniques:**
        Use algorithms that are robust to class imbalance, such as tree-based algorithms (Random Forest, Gradient Boosting) or ensemble methods.
        Adjust class weights in the model to penalize misclassification of minority classes more.

*    **Evaluation Metrics:**
        Choose appropriate evaluation metrics that are less sensitive to class imbalance, such as precision, recall, F1-score.



## Show a representative sample of data texts to find out required cleaning steps.

In [6]:
sample_data = dialects_data.sample(10)
sample_data

,id,text,dialect
36223,1145157943098257408,@marwaa1998 يا الله ويالله اهو قريب ربي يعطيك زي ماتحبي واكثر ميرو,LY
62991,901385889422733184,ليه بنحط لللبن ع الشاي؟\nعشان التفل يرضع \nكركركركركر,EG
24658,1001127971992887168,قناة #BBC كاتبة #الليبين يبيعونالمجوهرات من أجل قوت يومهم \nباهي اللي ما سمعوش بقصة #المغاطي الحمدالله ربي سترنا,LY
70175,1155430638197116928,@peteryoussef8 @shaarawy_eman كلنا مسئولين بصراحة \nمش بس الدولة \nطول عمرنا متربيين فى بيوت و مدارس سكتهم واحدة و طريقة تربيتهم واحدة \nاتربينا على المحبة و القيم و الاخلاق و حب الوطن \nده مش موجود دلوقتى,EG
120927,1184742070164045824,بدنا نرجع نتفق missed call واحد يعني وصلت\nاثنين missd call يعني عملت حادث وما وصلت,LB
14138,1149819383432302592,@bo_waell ان شاء الله تكون صعبة عليهم لانه لو صارت تجيني جلطة 😅 بس هم ممكن يبيعوا كوتينيو وديمبلي ويوفروا سيولة يشروا بها نيمار,LY
64103,1175189765186052096,@shamsshams0 لما نفضي هبقي أجيبلك كلامه وقتها,EG
6478,1158555441091293184,فلعادة منحبش فيديوهات نعمان لكن هذا ضحكني,LY
22337,1067515348365336576,@ManarSarhan مش اف سايد المؤشر غالط,LY
28649,693484107523842048,@libya2p0 @nadiahamza401والله رسمي خوي باسط سنة كاملة ف سواتر بيرالغنم والوطية ومشي لي امتحانات بس نجح ممتازهذا تو من يقنعه يمشي يكمل قرايته,LY


## Cleaning steps that we will need are:

- Remove Tatweel
- Remove digits and symbols
- Remove emojis
- Remove URLs
- Remove usernames
- Remove non-Arabic characters
- Remove extra spaces
- Remove hashtags
- Remove un-ASCII characters
- Stemming 
- Remove stop words
- Handling Noise and Garbage Characters
- Handling Bi-directional Text

## Split data into features and target

In [23]:
# Separate features (X) and target variable (y)
X_text = dialects_data['text']  # Text features
y = dialects_data['dialect']  # Target variable

In [42]:
X_text.head()

0                                                                                                                                                                                          @toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.
1                                                                                                                                                                                                                                   @AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة
2                                                                @smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند
3    @AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون

In [43]:
y.head()

0    LY
1    LY
2    LY
3    LY
4    LY
Name: dialect, dtype: object

## Split data into train,valid and test sets

In [44]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)


In [45]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25    , random_state=42)

In [46]:
X_train.shape, X_valid.shape, X_test.shape

((88630,), (29544,), (29544,))

## Define the custom transformer for Arabic text cleaning

In [70]:
class ArabicTextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, use_stemming=False):
        self.use_stemming = use_stemming
        self.arabic_stopwords = set(stopwords.words('arabic'))
        
        if self.use_stemming:
            self.stemmer = stemmer("arabic")
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # If X is a single string, convert it to a list of one element
        if isinstance(X, str):
            X = [X]
        return [self.clean_text(text) for text in X]
    
    def clean_text(self, text):
        print("Original text:", text)

        # Removing Non-Arabic Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
        print("After removing non-Arabic characters:", text)

        # Removing Tatweel
        text = text.replace('ـ', '')
        print("After removing Tatweel:", text)

        # Removing HTML Tags
        text = re.sub(r'<.*?>', '', text)
        print("After removing HTML tags:", text)

        # Tokenization
        words = word_tokenize(text)
        print("After tokenization:", words)

        # Removing Stopwords
        words = [word for word in words if word not in self.arabic_stopwords]
        print("After removing stopwords:", words)

        # Removing Digits and Symbols
        text = re.sub(r'[0-9’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+', '', ' '.join(words))
        print("After removing digits and symbols:", text)

        # Stemming
        if self.use_stemming:
            words = [self.stemmer.stemWord(word) for word in words]
        print("After stemming:", words)

        text = ' '.join(words)
        print("After joining words:", text)

        # Remove hashtag
        text = re.sub(r'#([^\s]+)', '', text)
        print("After removing hashtags:", text)

        # Removing URLs and Email Addresses and usernames
        text = re.sub(r'http\S+|www\S+|@\S+', '', text)
        print("After removing URLs and usernames:", text)

        # Removing Emojis and Symbols
        text = ''.join(char for char in text if not emoji.is_emoji(char))
        
        print("After removing emojis and symbols:", text)

        # Removing Repeated Characters
        text = re.sub(r'(.)\1{2,}', r'\1\1', text)
        print("After removing repeated characters:", text)

        # Handling Noise and Garbage Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Removing non-Arabic characters again
        print("After handling noise and garbage characters:", text)

       
        return text


In [74]:
# Try the ArabicTextCleaner class on the sample data
n="@smsm071990 @ALMOGRBE ♥️😂🥰👩🏻‍💻❎✅ كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند	"
cleaning = ArabicTextCleaner()
cleaned_n = cleaning.transform(n)
print(cleaned_n)

Original text: @smsm071990 @ALMOGRBE ♥️😂🥰👩🏻‍💻❎✅ كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
and they live happily ever after
ذي اند	
After removing non-Arabic characters:    كل  تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
     
ذي اند	
After removing Tatweel:    كل  تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
     
ذي اند	
After removing HTML tags:    كل  تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
     
ذي اند	
After tokenization: ['كل', 'تانيه', 'شاب', 'ليبي', 'بيرتاح', 'لبنت', 'مختلفة', 'ويلاحظ', 

In [76]:
class ArabicTextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, use_stemming=False):
        self.use_stemming = use_stemming
        self.arabic_stopwords = set(stopwords.words('arabic'))
        
        if self.use_stemming:
            self.stemmer = stemmer("arabic")
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # If X is a single string, convert it to a list of one element
        if isinstance(X, str):
            X = [X]
        return [self.clean_text(text) for text in X]
    
    def clean_text(self, text):
        
        # Removing Non-Arabic Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
        

        # Removing Tatweel
        text = text.replace('ـ', '')

        # Removing HTML Tags
        text = re.sub(r'<.*?>', '', text)

        # Tokenization
        words = word_tokenize(text)

        # Removing Stopwords
        words = [word for word in words if word not in self.arabic_stopwords]

        # Removing Digits and Symbols
        text = re.sub(r'[0-9’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+', '', ' '.join(words))

        # Stemming
        if self.use_stemming:
            words = [self.stemmer.stemWord(word) for word in words]

        text = ' '.join(words)


        # Remove hashtag
        text = re.sub(r'#([^\s]+)', '', text)

        # Removing URLs and Email Addresses and usernames
        text = re.sub(r'http\S+|www\S+|@\S+', '', text)
    

        # Removing Emojis and Symbols
        text = ''.join(char for char in text if not emoji.is_emoji(char))

        # Removing Repeated Characters
        text = re.sub(r'(.)\1{2,}', r'\1\1', text)
        
        # Handling Noise and Garbage Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Removing non-Arabic characters again

        return text

In [77]:
cleaning = ArabicTextCleaner()
cleaned_x_train = cleaning.transform(X_train)
cleaned_x_train[0]

'اي خبر او تطور جل الديب حدا يدقلي بفتحلو خط اغفى شوي'

In [79]:
cleaned_x_valid = cleaning.transform(X_valid)
cleaned_x_test = cleaning.transform(X_test)

In [85]:
class CustomTokenizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        
        self.tokenizer = Tokenizer()

    def fit(self, X, y=None):
        # Fit the tokenizer on the input text
        self.tokenizer.fit_on_texts(X)
        return self

    def transform(self, X, y=None):
       
        # Tokenize the input text into sequences of tokens
        sequences = self.tokenizer.texts_to_sequences(X)
        return sequences

In [86]:
# Instantiate an object of the CustomTokenizer class
tokenizer = CustomTokenizer()

# Fit the tokenizer on the training data
xtrain_tokens=tokenizer.fit(cleaned_x_train )


x_train_idx = tokenizer.transform(cleaned_x_train)
x_train_idx[0]

NameError: name 'Tokenizer' is not defined

# Calculate max lenth for sentence and determine VOCAB SIZE

In [ ]:
max_sequence_len = 0
for sentence in X_train:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

In [ ]:
import plotly.express as px

# Calculate lengths of sentences in x_train
sentence_lengths = [len(sentence) for sentence in x_train_idx]

# Create histogram figure using Plotly Express
fig = px.histogram(x=sentence_lengths, title='Distribution of Sentence Lengths in x_train_idx',
                   labels={'x': 'Sentence Length', 'y': 'Frequency'},
                  )
fig.show(

# Create a pipeline with the ArabicTextCleaner, TfidfVectorizer, and a classifier

In [80]:
pipeline = Pipeline([
    ('cleaner', ArabicTextCleaner(use_stemming=True)),
    ('vectorizer', TfidfVectorizer()),
    ('classifier', RandomForestClassifier())
])

In [84]:
pipeline.fit(cleaned_x_train , y_train)

KeyboardInterrupt: 

In [ ]:
validation_predictions = pipeline.predict(X_valid)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Compute precision, recall, and F1-score
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

----

##  Vectorize the text data using TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
X_text_vectorized = vectorizer.fit_transform(X_text)

## Apply SMOTE to generate synthetic samples

In [ ]:
# Instantiate SMOTE
smote = SMOTE(random_state=42)

In [ ]:
X_resampled, y_resampled = smote.fit_resample(X_text_vectorized, y)

In [ ]:

# Print class distribution before and after SMOTE
print("Class distribution before SMOTE:", Counter(y))
print("Class distribution after SMOTE:", Counter(y_resampled))

In [ ]:
# Create a DataFrame with resampled data
dialects_data_resampled = pd.DataFrame(X_resampled.todense(), columns=vectorizer.get_feature_names_out())
dialects_data_resampled['dialect'] = y_resampled


In [ ]:
# Plot the distribution of 'dialect' column after SMOTE
fig = px.histogram(dialects_data_resampled, x='dialect', color='dialect',
                   title='Distribution of Dialects after SMOTE',
                   labels={'dialect': 'Dialect', 'count': 'Frequency'},
                   color_discrete_sequence=px.colors.qualitative.Set1)
fig.update_xaxes(categoryorder='total descending')
fig.update_layout(xaxis_title="Dialect", yaxis_title="Frequency")
fig.show()

## show a representative sample of data texts to find out required preprocessing steps

In [ ]:
sample_data = dialects_data_resampled.sample(n=5)
sample_data